In [ ]:
import pandas as pd

df = pd.read_excel('EIDOS_taxonomia.xlsx')[['nameid', 'name', 'taxonrank', 'taxonomicgroup']]

In [112]:
df = df[df['taxonrank'].isin(['Species', 'Subspecies']) & (df['nameid'] < 100000)]
df['chosen'] = False
df['words'] = df['name'].str.split(' ').apply(lambda x: [word.replace('.', '').replace(',', '').replace('(', '').replace(')', '').lower() for word in x if len(word) > 2])
df['all_words'] = df['words'].apply(lambda x: set(x))
len(df)

89306

In [103]:
df.sample(5, random_state=0)

,nameid,name,taxonrank,taxonomicgroup,chosen,words,all_words,chosen_words
11560,11655.0,Lonchura malacca,Species,Aves,False,"[lonchura, malacca]","{lonchura, malacca}","{lonchura, malacca}"
8216,8224.0,Sedum rupestre L.,Species,Plantas vasculares,False,"[sedum, rupestre]","{rupestre, sedum}","{rupestre, sedum}"
58800,62432.0,Dicranota flammatra,Species,Invertebrados,False,"[dicranota, flammatra]","{dicranota, flammatra}","{dicranota, flammatra}"
81291,85058.0,Oscillatoria formosa,Species,Cromistas y Bacterias,False,"[oscillatoria, formosa]","{formosa, oscillatoria}","{formosa, oscillatoria}"
32506,36026.0,"Pseudosquillopsis cerisii (Roux, 1828)",Species,Invertebrados,False,"[pseudosquillopsis, cerisii, roux, 1828]","{pseudosquillopsis, cerisii, 1828, roux}","{pseudosquillopsis, cerisii, 1828, roux}"


In [ ]:
MIN_LENGTH = 3
ONLY_FULL_WORDS = True

all_words = set()
all_words_dict = {}
for i, row in df.iterrows():
    if not isinstance(row['words'], list):
        continue
    words_to_add = [word for word in row['words'] if len(word) > MIN_LENGTH - 1 and isinstance(word, str)]
    all_words.update(words_to_add)
    if not ONLY_FULL_WORDS:
        to_add = []
        for word in words_to_add:
            to_add += [word[:length+1] for length in range(len(word))]
        all_words.update(to_add)
        all_words_dict[i] = set(to_add)
    else:
        all_words_dict[i] = set(words_to_add)
df['all_words'] = df.index.map(all_words_dict)
df['chosen_words'] = df['all_words']
all_words = sorted(all_words, key=lambda x: len(x), reverse=True) # Las palabras mas largas primero
word_counts = pd.Series(
    [word for words in df['all_words'] if isinstance(words, set) for word in words]
).value_counts()
len(all_words)

68666

In [104]:
try:
    words_to_exclude = set(pd.read_csv('palabras_para_filtrar.csv', header=None)[0])
except:
    words_to_exclude = set()
df['chosen'] = df['all_words'].apply(lambda words: isinstance(words, set) and bool(words & words_to_exclude))
df['chosen_words'] = df['all_words']
df.loc[df['chosen'], 'chosen_words'] = [set() for _ in range((df['chosen']).sum())]
word_counts_only_chosen = pd.Series(
    [word for words in df['chosen_words'] if isinstance(words, set) for word in words]
).value_counts()
words_df = pd.DataFrame({'all': word_counts, 'chosen': word_counts_only_chosen}).fillna(0)
words_df['chosen'] = words_df['chosen'].astype(int)
words_df[words_df['all'] < 1000].sort_values('chosen', ascending=False)[:20]

,all,chosen
meigen,932,932
fabricius,808,808
fries,720,720
boiss,705,705
wollaston,700,700
1864,498,498
simon,497,497
canariensis,488,488
1838,473,473
persoon,436,436


In [110]:
import os

# Celda para comprobar cuantas especies hay descargadas en total
species_set = set()
for file in os.listdir('./IEPNB_filtrado'):
    sp = pd.read_excel(f'IEPNB_filtrado/{file}')
    species_set.update(sp['Código de la especie (id taxon)'])
len(species_set)

3404